In [6]:
# !pip install pandas
# !pip install sqlalchemy
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

from sqlalchemy import create_engine

In [7]:
# loading csv file as dataframe
dataframe = pd.read_csv("Amazon Sale Report.csv")

In [8]:
# checking first 5 rows of the dataframe
dataframe.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [9]:
# checking all column types
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

In [10]:
# checking unique values in Unnamed: 22 column
dataframe['Unnamed: 22'].unique()

array([nan, False], dtype=object)

In [11]:
# dropping unwanted index and 'Unnamed: 22' columns
dataframe = dataframe.drop(columns=['index','Unnamed: 22'], errors='ignore')

In [12]:
dataframe.columns

Index(['Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by'],
      dtype='object')

In [13]:
# lets drop duplicate rows in dataframe
dataframe.drop_duplicates(inplace=True)

In [14]:
dataframe = dataframe.reset_index(drop=True)

In [15]:
# checking null values in columns
dataframe.isnull().sum()

Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status         6872
Qty                       0
currency               7792
Amount                 7792
ship-city                33
ship-state               33
ship-postal-code         33
ship-country             33
promotion-ids         49150
B2B                       0
fulfilled-by          89692
dtype: int64

In [16]:
# FUnction to strip the spaces
def strip_spaces(df, column) -> dataframe:
    """
    Strips leading and trailing spaces from specified columns in a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to be cleaned.
    columns (list): A list of column names to strip spaces from.

    Returns:
    pd.DataFrame: The cleaned DataFrame with spaces stripped from specified columns.
    """
    if df[column].dtype == 'object':
        df[column] = df[column].str.strip()
    return df


### Cleaning Order ID

In [17]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Order ID')

In [18]:
dataframe['Order ID'].value_counts(ascending=True)

405-8078784-5731545     1
407-4776379-8226739     1
402-2059476-3753134     1
403-8280533-5594767     1
404-5453829-4022737     1
                       ..
408-3317403-1729937    10
403-0173977-3041148    11
404-9932919-6662730    11
403-4984515-8861958    12
171-5057375-2831560    12
Name: Order ID, Length: 120378, dtype: int64

In [19]:
# finding duplicate order id
duplicate_records = dataframe[dataframe.duplicated(subset=['Order ID'], keep=False)]
duplicate_records.head()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by
37,403-4367956-2849158,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3787,JNE3787-KR-S,kurta,S,...,1,INR,487.0,NEW DELHI,DELHI,110092.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
38,403-4367956-2849158,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3543,JNE3543-KR-S,kurta,S,...,1,INR,368.0,NEW DELHI,DELHI,110092.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
61,404-2262140-4696366,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE2270,JNE2270-KR-487-A-M,kurta,M,...,1,INR,518.0,NAGPUR,MAHARASHTRA,440022.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
62,404-2262140-4696366,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3579,JNE3579-KR-M,kurta,M,...,1,INR,295.0,NAGPUR,MAHARASHTRA,440022.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN
79,408-4069830-3819562,04-30-22,Shipped,Amazon,Amazon.in,Expedited,SET288,SET288-KR-NP-XXXL,Set,3XL,...,1,INR,684.0,Kolkata,WEST BENGAL,700048.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN


In [20]:
''' dropping all the duplicate records by keeping last one assuming all previous 
    records are overwritten by last record
''' 
dataframe = dataframe.sort_values(by=['Order ID', 'Date'], ascending=[True, False])
dataframe = dataframe.drop_duplicates(subset=['Order ID'], keep='first')

In [21]:
dataframe['Order ID'].value_counts(ascending=True)

171-0000547-8192359    1
406-4600662-7808350    1
406-4600557-0040364    1
406-4600542-6625166    1
406-4600522-4517153    1
                      ..
403-7247303-0806739    1
403-7245456-5809968    1
403-7245319-8594743    1
403-7252648-2824327    1
S02-9878098-5959538    1
Name: Order ID, Length: 120378, dtype: int64

In [22]:
# checking if Order ID have only alphabetic characters
dataframe[dataframe['Order ID'].str.isalpha()]

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by


### Cleaning Date Field

In [23]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Date')

In [24]:
# checking if Date have only alphabetic characters
dataframe[dataframe['Date'].str.isalpha()]

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by


In [25]:
dataframe['Date'].value_counts(ascending=True)

03-31-22     158
06-29-22     606
06-24-22     950
06-25-22     977
05-21-22    1002
            ... 
04-15-22    1733
04-14-22    1754
05-04-22    1896
05-02-22    1906
05-03-22    1941
Name: Date, Length: 91, dtype: int64

In [26]:
# typecasting date from object format to date format
dataframe['Date'] = pd.to_datetime(dataframe['Date'], format='%m-%d-%y')

### Cleaning Status

In [27]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Status')

In [28]:
# checking unique values in Status column
dataframe['Status'].unique()

array(['Shipped', 'Cancelled', 'Shipped - Delivered to Buyer', 'Pending',
       'Pending - Waiting for Pick Up', 'Shipped - Returned to Seller',
       'Shipped - Picked Up', 'Shipped - Returning to Seller',
       'Shipped - Out for Delivery', 'Shipped - Rejected by Buyer',
       'Shipped - Lost in Transit', 'Shipped - Damaged', 'Shipping'],
      dtype=object)

### Cleaning Fulfilment

In [29]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Fulfilment')

In [30]:
# checking unique values in Fulfilment column
dataframe['Fulfilment'].unique()

array(['Amazon', 'Merchant'], dtype=object)

### Cleaning Sales Channel

In [31]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Sales Channel ')

In [32]:
# checking unique values in Status column
dataframe['Sales Channel '].unique()

array(['Amazon.in', 'Non-Amazon'], dtype=object)

### Cleaning ship-service-level

In [33]:
# string while spaces
dataframe = strip_spaces(dataframe, 'ship-service-level')

In [34]:
# checking unique values in ship-service-level column
dataframe['ship-service-level'].unique()

array(['Expedited', 'Standard'], dtype=object)

### Cleaning Style

In [35]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Style')
# checking unique values in style column
dataframe['Style'].unique()

array(['JNE2032', 'J0097', 'JNE3440', ..., 'J0324', 'SET072', 'JNE2058'],
      dtype=object)

In [36]:
# checking if Style have only alphabetic characters to check any NAN/NA/Null values
dataframe[dataframe['Style'].str.isalpha()]

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by


### Cleaning SKU

In [37]:
# string while spaces
dataframe = strip_spaces(dataframe, 'SKU')
# checking unique values in ship-service-level column
dataframe['SKU'].unique()

array(['JNE2032-KR-205-XL', 'J0097-KR-M', 'JNE3440-KR-N-XS', ...,
       'JNE3225-KR-XXXL', 'J0100-DR-S', 'BTM030-NP-S'], dtype=object)

In [38]:
# checking if SKU have only alphabetic characters to check any NAN/NA/Null values
dataframe[dataframe['SKU'].str.isalpha()]

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by


### Cleaning Category

In [39]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Category')
# checking unique values in Category column
dataframe['Category'].unique()

array(['kurta', 'Set', 'Top', 'Blouse', 'Western Dress', 'Ethnic Dress',
       'Bottom', 'Saree', 'Dupatta'], dtype=object)

### Cleaning Size

In [40]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Size')

# checking unique values in Size column
dataframe['Size'].unique()

array(['XL', 'M', 'XS', 'L', '3XL', 'XXL', 'S', 'Free', '5XL', '4XL',
       '6XL'], dtype=object)

### Cleaning ASIN

In [41]:
# string while spaces
dataframe = strip_spaces(dataframe, 'ASIN')

# checking unique values in ASIN column
dataframe['ASIN'].unique()

array(['B0768J7VQ1', 'B08BJT3PSM', 'B09HMY3YLT', ..., 'B07MY19YD4',
       'B08VS4RLVT', 'B0845TNXLX'], dtype=object)

### Cleaning Courier Status

In [42]:
# string while spaces
dataframe = strip_spaces(dataframe, 'Courier Status')
# checking unique values in Courier Status
dataframe['Courier Status'].unique()

array(['Shipped', 'Unshipped', nan, 'Cancelled'], dtype=object)

### Cleaning Qty

In [43]:
# checking unique values in Qty
dataframe['Qty'].unique()

array([ 1,  0,  2,  3,  4,  5,  9, 15, 13])

### Cleaning currency

In [44]:
# removing spaces at starting and end of currency if anything is there and checking unque values
dataframe = strip_spaces(dataframe, 'currency')
dataframe['currency'].unique()

array(['INR', nan], dtype=object)

### Cleaning Amount

In [45]:
# checking unique values in Amount
dataframe['Amount'].unique()

array([ 301.  ,  544.  ,  422.  , ...,  487.62, 2397.  , 2864.  ])

### Cleaning ship-city 

In [46]:
# removing spaces at starting and end of ship-city if anything is there and checking unque values
dataframe = strip_spaces(dataframe, 'ship-city')
dataframe['ship-city'].unique()

array(['PIMPRI CHINCHWAD', 'Howrah', 'KODAD', ..., 'jalgaon',
       'NEW DELHIDe', 'District Fatehabad'], dtype=object)

In [48]:
def classify_city(city, major_cities):
    if isinstance(city, str):
        # Clean the city name
        cleaned_city = re.sub(r'[^\w\s]', '', city)  # Remove special characters except spaces
        cleaned_city = re.sub(r'\d+', '', cleaned_city)  # Remove numbers
        cleaned_city = cleaned_city.strip()  # Remove leading and trailing spaces

        # Convert cleaned city name to title case
        cleaned_city_title = cleaned_city.title()

        # Check if any major city is part of the cleaned city name
        for major_city in major_cities:
            if major_city.lower() in cleaned_city_title.lower():
                return major_city

    return 'Other'

# Example usage
major_cities = [
    'Mumbai', 'Delhi', 'Bangalore', 'Kolkata', 'Chennai', 'Hyderabad', 'Pune', 
    'Ahmedabad', 'Surat', 'Jaipur', 'Lucknow', 'Kanpur', 'Nagpur', 'Indore', 
    'Bhopal', 'Visakhapatnam', 'Vadodara', 'Ghaziabad', 'Ludhiana', 'Agra', 
    'Nashik', 'Meerut', 'Faridabad', 'Ranchi', 'Jabalpur', 'Kota', 'Udaipur', 
    'Coimbatore', 'Madurai', 'Thane', 'Mangalore', 'Jodhpur', 'Amritsar', 
    'Navi Mumbai', 'Dehradun', 'Shimla', 'Gangtok', 'Trivandrum', 'Tiruchirappalli', 
    'Salem', 'Warangal', 'Raipur', 'Kakinada', 'Bhubaneswar', 'Puducherry', 
    'Chandigarh', 'Durgapur', 'Bhilai', 'Siliguri', 'Tirupati', 'Kollam', 
    'Jamshedpur', 'Bilaspur', 'Hapur', 'Ambala', 'Satna', 'Aurangabad', 
    'Mysore', 'Hubli', 'Nellore', 'Gwalior', 'Sambalpur', 'Rourkela', 
    'Vellore', 'Erode', 'Karaikal', 'Davanagere', 'Kochi', 'Kottayam', 
    'Malappuram', 'Pathanamthitta', 'Kannur', 'Alappuzha', 'Kasaragod', 
    'Chennai', 'Tirunelveli', 'Tuticorin', 'Nagercoil', 'Ramanathapuram', 
    'Dharmapuri', 'Salem', 'Vellore'
]

In [49]:
dataframe['classified_city'] = dataframe['ship-city'].apply(lambda x: classify_city(x, major_cities))

In [50]:
dataframe['ship-city'] = dataframe['ship-city'].str.title()

### Cleaning ship-state

In [51]:
# removing spaces at starting and end of ship-state if anything is there and checking unque values
dataframe = strip_spaces(dataframe, 'ship-state')
dataframe['ship-state'].unique()

array(['MAHARASHTRA', 'WEST BENGAL', 'TELANGANA', 'UTTAR PRADESH',
       'DELHI', 'punjab', 'BIHAR', 'Gujarat', 'JHARKHAND', 'TRIPURA',
       'HARYANA', 'RAJASTHAN', 'ANDHRA PRADESH', 'ASSAM', 'KARNATAKA',
       'UTTARAKHAND', 'TAMIL NADU', 'KERALA', 'ODISHA', 'CHHATTISGARH',
       'New Delhi', 'MADHYA PRADESH', 'HIMACHAL PRADESH',
       'JAMMU & KASHMIR', 'PUDUCHERRY', 'GOA', 'MANIPUR', 'MEGHALAYA',
       'Delhi', 'PUNJAB', 'SIKKIM', 'CHANDIGARH', 'MIZORAM',
       'ARUNACHAL PRADESH', 'Goa', 'DADRA AND NAGAR', 'ANDAMAN & NICOBAR',
       'Chandigarh', 'delhi', 'Rajasthan', 'NL', 'Orissa', 'NAGALAND',
       'Punjab', 'Odisha', 'LADAKH', nan, 'Bihar', 'rajasthan', 'Sikkim',
       'rajsthan', 'orissa', 'goa', 'Manipur', 'Pondicherry',
       'Arunachal Pradesh', 'LAKSHADWEEP', 'Meghalaya',
       'Arunachal pradesh', 'Nagaland', 'PB', 'Rajshthan', 'Mizoram',
       'APO', 'Rajsthan', 'RJ', 'AR', 'bihar', 'Puducherry',
       'Punjab/Mohali/Zirakpur'], dtype=object)

In [52]:
dataframe['ship-state'] = dataframe['ship-state'].str.title()

In [53]:
# Create a mapping dictionary to correct state names
state_mapping = {
    'Puducherry': 'Pondicherry',
    'Tamil Nadu': 'Tamil Nadu',
    'Uttar Pradesh': 'Uttar Pradesh',
    'Chandigarh': 'Chandigarh',
    'Telangana': 'Telangana',
    'Andhra Pradesh': 'Andhra Pradesh',
    'Rajasthan': 'Rajasthan',
    'Delhi': 'Delhi',
    'Haryana': 'Haryana',
    'Assam': 'Assam',
    'Jharkhand': 'Jharkhand',
    'Chhattisgarh': 'Chhattisgarh',
    'Odisha': 'Odisha',
    'Kerala': 'Kerala',
    'Madhya Pradesh': 'Madhya Pradesh',
    'West Bengal': 'West Bengal',
    'Nagaland': 'Nagaland',
    'Gujarat': 'Gujarat',
    'Uttarakhand': 'Uttarakhand',
    'Bihar': 'Bihar',
    'Jammu & Kashmir': 'Jammu & Kashmir',
    'Punjab': 'Punjab',
    'Himachal Pradesh': 'Himachal Pradesh',
    'Arunachal Pradesh': 'Arunachal Pradesh',
    'Manipur': 'Manipur',
    'Goa': 'Goa',
    'Meghalaya': 'Meghalaya',
    'Tripura': 'Tripura',
    'Ladakh': 'Ladakh',
    'Dadra And Nagar': 'Dadra and Nagar Haveli',
    'Sikkim': 'Sikkim',
    'Andaman & Nicobar': 'Andaman and Nicobar Islands',
    'Rajshthan': 'Rajasthan',          # Correction for misspelled 'Rajasthan'
    'Nl': 'Nagaland',                  # Assuming 'Nl' is 'Nagaland'
    'Mizoram': 'Mizoram',
    'New Delhi': 'Delhi',              # Treat 'New Delhi' as 'Delhi'
    'Punjab/Mohali/Zirakpur': 'Punjab',# Treat as 'Punjab'
    'Rj': 'Rajasthan',                 # Correction for 'Rajasthan'
    'Orissa': 'Odisha',                # Older name for 'Odisha'
    'Lakshadweep': 'Lakshadweep',
    'Pb': 'Punjab',                    # Abbreviation for 'Punjab'
    'Apo': 'Andhra Pradesh',           # Assuming 'Apo' might be a typo for 'Andhra Pradesh'
    'Ar': 'Arunachal Pradesh',         # Assuming 'Ar' is 'Arunachal Pradesh'
    'Pondicherry': 'Pondicherry',      # Alternate spelling for 'Puducherry'
    'Rajsthan': 'Rajasthan'            # Correction for misspelled 'Rajasthan'
}

dataframe['ship-state'] = dataframe['ship-state'].replace(state_mapping)

In [54]:
dataframe['ship-state'].unique()

array(['Maharashtra', 'West Bengal', 'Telangana', 'Uttar Pradesh',
       'Delhi', 'Punjab', 'Bihar', 'Gujarat', 'Jharkhand', 'Tripura',
       'Haryana', 'Rajasthan', 'Andhra Pradesh', 'Assam', 'Karnataka',
       'Uttarakhand', 'Tamil Nadu', 'Kerala', 'Odisha', 'Chhattisgarh',
       'Madhya Pradesh', 'Himachal Pradesh', 'Jammu & Kashmir',
       'Pondicherry', 'Goa', 'Manipur', 'Meghalaya', 'Sikkim',
       'Chandigarh', 'Mizoram', 'Arunachal Pradesh',
       'Dadra and Nagar Haveli', 'Andaman and Nicobar Islands',
       'Nagaland', 'Ladakh', nan, 'Lakshadweep'], dtype=object)

### Cleaning ship-postal-code

In [55]:
# checking unque values
dataframe['ship-postal-code'].unique()

array([412101., 711104., 508206., ..., 533343., 502205., 679123.])

### Cleaning ship-country 

In [56]:
# removing spaces at starting and end of ship-country if anything is there and checking unque values
dataframe = strip_spaces(dataframe, 'ship-country')
dataframe['ship-country'].unique()

array(['IN', nan], dtype=object)

In [57]:
dataframe['ship-country'] = dataframe['ship-country'].replace({'IN':'India'})

### Cleaning promotion-ids

In [58]:
# removing spaces at starting and end of promotion-ids if anything is there and checking unque values
dataframe = strip_spaces(dataframe, 'promotion-ids')
dataframe['promotion-ids'].unique()

array([nan, 'IN Core Free Shipping 2015/04/08 23-48-5-108',
       'Amazon PLCC Free-Financing Universal Merchant AAT-WNKTBO3K27EJC,Amazon PLCC Free-Financing Universal Merchant AAT-QX3UCCJESKPA2,Amazon PLCC Free-Financing Universal Merchant AAT-5QQ7BIYYQEDN2,Amazon PLCC Free-Financing Universal Merchant AAT-DSJ2QRXXWXVMQ,Amazon PLCC Free-Financing Universal Merchant AAT-CXJHMC2YJUK76,Amazon PLCC Free-Financing Universal Merchant AAT-SL3Y2YRMAUNFC,Amazon PLCC Free-Financing Universal Merchant AAT-CC4FAVTYR4X7C,Amazon PLCC Free-Financing Universal Merchant AAT-XXRCW6NZEPZI4,Amazon PLCC Free-Financing Universal Merchant AAT-R7GXNZWISTRFA,Amazon PLCC Free-Financing Universal Merchant AAT-EOKPWFWYW7Y6I,Amazon PLCC Free-Financing Universal Merchant AAT-N6QGG4HRHN6T4,Amazon PLCC Free-Financing Universal Merchant AAT-PZHUW6DZX4BPS,Amazon PLCC Free-Financing Universal Merchant AAT-ETXQ3XXWMRXBG,Amazon PLCC Free-Financing Universal Merchant AAT-7X3XCTYG64VBE,Amazon PLCC Free-Financing Universal

### Cleaning B2B

In [59]:
# checking unique values in Size column
dataframe['B2B'].unique()

array([False,  True])

### Cleaning fulfilled-by

In [60]:
# removing spaces at starting and end of fulfilled-by if anything is there
dataframe = strip_spaces(dataframe, 'fulfilled-by')

# checking unique values in Size column
dataframe['fulfilled-by'].unique()

array([nan, 'Easy Ship'], dtype=object)

### Renaming Column Names

In [61]:
dataframe.columns

Index(['Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'classified_city'],
      dtype='object')

In [62]:
# Rename columns: convert to lowercase, replace spaces with underscores, and replace hyphens with underscores
dataframe.columns = dataframe.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

In [63]:
dataframe.columns

Index(['order_id', 'date', 'status', 'fulfilment', 'sales_channel_',
       'ship_service_level', 'style', 'sku', 'category', 'size', 'asin',
       'courier_status', 'qty', 'currency', 'amount', 'ship_city',
       'ship_state', 'ship_postal_code', 'ship_country', 'promotion_ids',
       'b2b', 'fulfilled_by', 'classified_city'],
      dtype='object')

In [64]:
dataframe.rename(columns={'sales_channel_':'sales_channel'}, inplace=True)

## Exporting data to Database

In [65]:
db_type = 'postgresql'
username = 'fedex_user'
password = "Fedex9663"
host = 'localhost'
port = '5432'  # default port for PostgreSQL, change if using a different database
database_name = 'fedex'

# Create a database connection string
connection_string = f"{db_type}://{username}:{password}@{host}:{port}/{database_name}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to the database with table name 'cleaned_dataset'
dataframe.to_sql('cleaned_dataset', engine, if_exists='replace', index=False)

print("Data successfully loaded into the database.")


Data successfully loaded into the database.


In [66]:
dataframe.columns

Index(['order_id', 'date', 'status', 'fulfilment', 'sales_channel',
       'ship_service_level', 'style', 'sku', 'category', 'size', 'asin',
       'courier_status', 'qty', 'currency', 'amount', 'ship_city',
       'ship_state', 'ship_postal_code', 'ship_country', 'promotion_ids',
       'b2b', 'fulfilled_by', 'classified_city'],
      dtype='object')

In [347]:
status: ['Shipped', 'Cancelled', 'Shipped - Delivered to Buyer', 'Pending',
       'Pending - Waiting for Pick Up', 'Shipped - Returned to Seller',
       'Shipped - Picked Up', 'Shipped - Returning to Seller',
       'Shipped - Out for Delivery', 'Shipped - Rejected by Buyer',
       'Shipped - Lost in Transit', 'Shipped - Damaged', 'Shipping']
fulfilment: ['Amazon', 'Merchant']
sales_channel: ['Amazon.in', 'Non-Amazon']
ship_service_level: ['Expedited', 'Standard']
courier_status: ['Shipped', 'Unshipped', nan, 'Cancelled']
b2b: [True, False]
fulfilled_by: [nan, 'Easy Ship']